In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [2]:
with open('./all_docs_results.json') as json_file:
    predictions = json.load(json_file)
    
with open('./project-12-at-2021-07-15-14-39-0020e5b8.json') as json_file:
    ground_truth = json.load(json_file)

## Predictions
### Exploración

In [3]:
predictions['11821']['CSF_match_results'].keys()

dict_keys(['state_csf', 'society_type_csf', 'company_name_csf'])

In [4]:
predictions['11821']['ROIS_qa_results'].keys()

dict_keys(['notario', 'date'])

In [5]:
society_type_csf = predictions['11821']['CSF_match_results']['society_type_csf'][3]
company_name_csf = predictions['11821']['CSF_match_results']['company_name_csf'][3]
notario = predictions['11821']['ROIS_qa_results']['notario'][3]
date = predictions['11821']['ROIS_qa_results']['date'][3]

## Armo un df de predicciones

In [6]:
campos = []
for i in range(len(predictions)):
    for k in predictions['11821']['CSF_match_results'].keys():
        campos.append(k)
    for k in predictions['11821']['ROIS_qa_results'].keys():
        campos.append(k)
        
campos_unique = sorted(set(campos))
campos_unique

['company_name_csf', 'date', 'notario', 'society_type_csf', 'state_csf']

In [7]:
df_preds = pd.DataFrame(columns = ['documento'])
for c in campos_unique:
    df_preds[c] = []
    
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf


In [8]:
i = 0
for k in predictions.keys():
    
    df_preds.loc[i, 'documento'] = k
    
    # a veces puede ser que 'CSF_match_results' o 'ROIS_qa_results' estén vacíos
    try:
        for j in predictions[k]['CSF_match_results'].keys():
            try:
                df_preds.loc[i, j] = predictions[k]['CSF_match_results'][j][3]
            except:
                continue
    except:
        continue
        
    try:    
        for j in predictions[k]['ROIS_qa_results'].keys():
            try:
                df_preds.loc[i, j] = predictions[k]['ROIS_qa_results'][j][3]
            except:
                continue
    except:
        continue
        
    i = i+1
            
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,GRUPO CYCMP,veintiumo de julio del dos mil ocho,MAURICIO MIER PADRÓN,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,San Luis Potosi
1,12078,BITUBI CONSULTING,trece de febrero,JORGE JULIÁN PEREYRA ESCUDERO,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,Quintana Roo
2,12192,DESARROLLO DE SOFTWARE MONOBITS,22 de noviembre de 2012,YO,Sociedad de Responsabilidad Limitada de Capita...,BAJA CALIFORNIA
3,12264,CHARVEL Y PEZUELA GROUP,dos de octubre,NaN,SOCIEDAD ANONIMA DE CAPITAL VARIABLE,CIUDAD DE MÉXICO
4,12286,EXELENCIA EJECUTIVA PREMIUM,NaN,RAFAEL OVARRUBIAS FLORES,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,JALISCO


De todos los campos que puedo encontrar en las actas, no todos los puedo validar, sólo éstos de arriba

## Ground truth
### Exploración

In [9]:
# este tiene la boleta de inscripción
ground_truth[0]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.866666666666667,
   'y': 11.902834008097166,
   'width': 83.46666666666667,
   'height': 36.51821862348178,
   'rotation': 0,
   'rectanglelabels': ['boleta_inscripcion']},
  'id': 'nVvt8xx7hA',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'choices': ['yes']},
  'id': 'r_7Rb71M-M',
  'from_name': 'has_boleta_inscripcion',
  'to_name': 'image',
  'type': 'choices'}]

In [10]:
# este tiene el cuadro accionario y el apoderado
ground_truth[1]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.432706222865413,
   'y': 39.19156414762742,
   'width': 57.45296671490593,
   'height': 7.0298769771529,
   'rotation': 0,
   'rectanglelabels': ['sh_table_lines']},
  'id': 'ANsPu48gOp',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'text': ['Jorge Cañedo Osuna']},
  'id': 'asa879Jesq',
  'from_name': 'apoderado',
  'to_name': 'image',
  'type': 'textarea'}]

In [11]:
# este tiene varios campos
# lo que me interesa es todo lo que está en "result"
result_lista = ground_truth[2]['annotations'][0]['result']

result_lista

[{'value': {'text': ['Miguel Enrique Martinez Serrato']},
  'id': 'KdMY7vBSEi',
  'from_name': 'notario',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Compass4PL']},
  'id': 'hQFu6ebrse',
  'from_name': 'razon_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['S de RL de cv']},
  'id': 'P7GI5OarC-',
  'from_name': 'denominacion_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['27/06/2015']},
  'id': '_LjHuXTpo8',
  'from_name': 'fecha_inscripcion',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['12236']},
  'id': 'cSq9FwAKry',
  'from_name': 'instrumento_publico',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Navojoa Sonora Mexico']},
  'id': 'XAR0_Uu-DH',
  'from_name': 'direccion',
  'to_name': 'image',
  'type': 'textarea'}]

In [12]:
id = ground_truth[1]['id']
id

7887

In [13]:
documento = ground_truth[1]['file_upload'].split('-')[0]
documento

'16519'

In [14]:
# la página a veces tiene un "_" seguido de otras cosas
pagina = ground_truth[1]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
pagina

'014'

In [15]:
# acá no hay campos que me interesen
ground_truth[0]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.866666666666667,
   'y': 11.902834008097166,
   'width': 83.46666666666667,
   'height': 36.51821862348178,
   'rotation': 0,
   'rectanglelabels': ['boleta_inscripcion']},
  'id': 'nVvt8xx7hA',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'choices': ['yes']},
  'id': 'r_7Rb71M-M',
  'from_name': 'has_boleta_inscripcion',
  'to_name': 'image',
  'type': 'choices'}]

In [16]:
# acá me interesa el campo "apoderado"
ground_truth[1]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.432706222865413,
   'y': 39.19156414762742,
   'width': 57.45296671490593,
   'height': 7.0298769771529,
   'rotation': 0,
   'rectanglelabels': ['sh_table_lines']},
  'id': 'ANsPu48gOp',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'text': ['Jorge Cañedo Osuna']},
  'id': 'asa879Jesq',
  'from_name': 'apoderado',
  'to_name': 'image',
  'type': 'textarea'}]

Vemos que el primer key es "value".

Acá hay varios campos que podría levantar, aunque no son los 4 que tengo en el otro archivo. También empiezan con "value":

In [17]:
ground_truth[2]['annotations'][0]['result']

[{'value': {'text': ['Miguel Enrique Martinez Serrato']},
  'id': 'KdMY7vBSEi',
  'from_name': 'notario',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Compass4PL']},
  'id': 'hQFu6ebrse',
  'from_name': 'razon_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['S de RL de cv']},
  'id': 'P7GI5OarC-',
  'from_name': 'denominacion_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['27/06/2015']},
  'id': '_LjHuXTpo8',
  'from_name': 'fecha_inscripcion',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['12236']},
  'id': 'cSq9FwAKry',
  'from_name': 'instrumento_publico',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Navojoa Sonora Mexico']},
  'id': 'XAR0_Uu-DH',
  'from_name': 'direccion',
  'to_name': 'image',
  'type': 'textarea'}]

## Armo un df de ground truth

In [18]:
# quiero saber cuáles son todos los campos posibles para hacer columnas en el df
campos = []
for i in range(len(ground_truth)):
    lista = ground_truth[i]['annotations'][0]['result']
    
    for l in lista:
        # acá salteo todos los que no tengan el formato que me interesa
        try:
            campos.append(l['from_name'])
        except:
            continue
            
campos_unique = sorted(set(campos))

In [19]:
for c in campos_unique:
    print(c)

apoderado
denominacion_social
direccion
fecha_inscripcion
has_boleta_inscripcion
instrumento_publico
label
notario
razon_social


In [20]:
# hago una nueva versión del df con una columna por campo
df_truth = pd.DataFrame(columns = ['id', 'documento', 'pagina'])
for c in campos_unique:
    df_truth[c] = []
    
# df.head()

In [21]:
# ahora trato de llenar todos esos campos, cuando es posible

for i in range(len(ground_truth)):
#     print(i)
    identity = ground_truth[i]['id']
    documento = ground_truth[i]['file_upload'].split('-')[0]
    pagina = ground_truth[i]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
    
    lista = ground_truth[i]['annotations'][0]['result']

    for l in lista:
        df_truth.loc[i, 'id'] = identity
        df_truth.loc[i, 'documento'] = documento
        df_truth.loc[i, 'pagina'] = pagina
        
        try:
            df_truth.loc[i, l['from_name']] = l['value']['text'][0]
        except:
            continue
    
# df_truth.head()

In [87]:
# ordeno por documento y página
df_truth = df_truth.sort_values(['documento', 'pagina'])
df_truth.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
1276,6631,10227,009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,6635,10227,019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,6908,11821,000,NaN,sa de cv,san luis potosi,21/07/2008,NaN,39413,NaN,mauricio mier padron,grupo cycme
95,6897,11821,011,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,6910,11821,015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# convierto pagina a int
df_truth['pagina'] = df_truth['pagina'].astype(int)
df_truth.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
1276,6631,10227,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,6635,10227,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,6908,11821,0,NaN,sa de cv,san luis potosi,21/07/2008,NaN,39413,NaN,mauricio mier padron,grupo cycme
95,6897,11821,11,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,6910,11821,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Comparación entre los 2 dataframes
Hago un df nuevo donde dejo sólo los campos que quiero comparar, y colapso todas las páginas.

Quizás es más fácil construirlo directamente de esa forma. En este caso los campos "id" y "pagina" ya no tienen sentido.

In [23]:
# este df lo lleno juntando todas las páginas
# para eso necesito iterar por documento. Hago una lista de todos los docs que hay

docs = []
for i in range(len(ground_truth)):
    documento = ground_truth[i]['file_upload'].split('-')[0]
    docs.append(documento)
    
docs_unique = sorted(list(set(docs)))

In [24]:
df_truth_compact = pd.DataFrame(docs_unique, columns = ['documento'])
# df_truth_compact.head()

In [25]:
# i=0
for d in docs_unique:
    for g in ground_truth:
        documento = g['file_upload'].split('-')[0]
        if documento == d:
            lista = g['annotations'][0]['result']

            for l in lista:
                try:
                    df_truth_compact.loc[df_truth_compact['documento'] == d, l['from_name']] = l['value']['text'][0]
#                     print(i, documento, l['from_name'], l['value']['text'][0])
                except:
                    continue
#             i=i+1
            
df_truth_compact.head()

,documento,apoderado,notario,razon_social,denominacion_social,direccion,fecha_inscripcion,instrumento_publico
0,10227,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11821,raymundo garza elizondo,mauricio mier padron,grupo cycme,sa de cv,san luis potosi,21/07/2008,39413
2,12078,mauricio zúñiga de la cerda,jorge julian pereyra escudero,bitubi consulting,sa de cv,cancun quintana roo,27/03/2018,360
3,12192,cesar higuera chavez,virgilio cesar valdez portales,desarrollo de software monobits,s de rl de cv,"mexicali, baja california",21/01/2016,107826
4,12264,lilia guadalupe gomez pezuela gamboa,Jorge Rábago Ordóñez,charvel y pezuela group,sa de cv,"ciudad de mexico, distrito federal",02/10/2012,437


Me quiero quedar sólo con las columnas que voy a poder validar:

In [26]:
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,GRUPO CYCMP,veintiumo de julio del dos mil ocho,MAURICIO MIER PADRÓN,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,San Luis Potosi
1,12078,BITUBI CONSULTING,trece de febrero,JORGE JULIÁN PEREYRA ESCUDERO,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,Quintana Roo
2,12192,DESARROLLO DE SOFTWARE MONOBITS,22 de noviembre de 2012,YO,Sociedad de Responsabilidad Limitada de Capita...,BAJA CALIFORNIA
3,12264,CHARVEL Y PEZUELA GROUP,dos de octubre,NaN,SOCIEDAD ANONIMA DE CAPITAL VARIABLE,CIUDAD DE MÉXICO
4,12286,EXELENCIA EJECUTIVA PREMIUM,NaN,RAFAEL OVARRUBIAS FLORES,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,JALISCO


In [27]:
df_truth_compact = df_truth_compact[['documento', 'razon_social', 'fecha_inscripcion', 'notario', 'denominacion_social', 'direccion']]
df_truth_compact.head()

,documento,razon_social,fecha_inscripcion,notario,denominacion_social,direccion
0,10227,NaN,NaN,NaN,NaN,NaN
1,11821,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
2,12078,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
3,12192,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
4,12264,charvel y pezuela group,02/10/2012,Jorge Rábago Ordóñez,sa de cv,"ciudad de mexico, distrito federal"


### Normalizo el texto de las predicciones
Lower case, sin acentos.

In [28]:
# convierto todo a lower case
for col in list(df_preds.drop(['documento'], axis=1)):
    df_preds[col] = df_preds[col].str.lower()
    
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padrón,sociedad anónima de capital variable,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julián pereyra escudero,sociedad anónima de capital variable,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,sociedad de responsabilidad limitada de capita...,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sociedad anonima de capital variable,ciudad de méxico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sociedad anónima de capital variable,jalisco


In [29]:
# elimino acentos
for col in list(df_preds.drop(['documento'], axis=1)):
    df_preds[col] = df_preds[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sociedad anonima de capital variable,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sociedad anonima de capital variable,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,sociedad de responsabilidad limitada de capita...,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sociedad anonima de capital variable,ciudad de mexico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sociedad anonima de capital variable,jalisco


Se me ocurre que society type se puede estandarizar a mano (debe haber pocos valores)

df_truth_compact también tiene mayúsculas:

In [30]:
# convierto todo a lower case
for col in list(df_truth_compact.drop(['documento'], axis=1)):
    df_truth_compact[col] = df_truth_compact[col].str.lower()

In [31]:
df_preds['society_type_csf'].value_counts()

sociedad anonima de capital variable                           11
sociedad de responsabilidad limitada de capital variable        2
sociedad anonima promotora de inversion de capital variable     1
Name: society_type_csf, dtype: int64

In [32]:
# veo cuáles son las otras abreviaciones
df_truth_compact['denominacion_social'].value_counts()

sa de cv                22
s de rl de cv            3
scp                      1
sapi de cv sofom enr     1
sas de cv                1
Name: denominacion_social, dtype: int64

In [33]:
df_preds = df_preds.replace(['sociedad anonima de capital variable'],'sa de cv')
df_preds = df_preds.replace(['sociedad de responsabilidad limitada de capital variable'],'s de rl de cv')
df_preds = df_preds.replace(['sociedad anonima promotora de inversion de capital variable'],'sapi de cv')

df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco


In [34]:
df_truth_compact.head()

,documento,razon_social,fecha_inscripcion,notario,denominacion_social,direccion
0,10227,NaN,NaN,NaN,NaN,NaN
1,11821,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
2,12078,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
3,12192,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
4,12264,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"


## Estandarizar fechas
Veo qué pinta tienen

In [35]:
df_preds['date'].value_counts()

22 de noviembre de 2012                  2
septiembre                               1
mayos                                    1
13 trece de octubre de 2018 dieciocho    1
once de julio del ano                    1
trece de febrero                         1
doce de septiembre                       1
veintiumo de julio del dos mil ocho      1
dos de octubre                           1
Name: date, dtype: int64

No sé ni por dónde empezar. Capaz puedo empezar matcheando otras cosas. Denominacion_social es fácil.

Creo que me conviene hacer un join de los dos df, para recorrer todos los documentos. Sería un inner join, porque me interesan sólo los documentos que están presentes en ambos datasets.

### Estandarizo los nombres de las columnas:

In [36]:
df_preds.rename(columns = {'date':'date_csf', 'notario':'notario_csf'}, inplace = True)
df_truth_compact.rename(columns = {'razon_social':'company_name_pred', 'fecha_inscripcion':'date_pred', 'notario':'notario_pred', 'denominacion_social':'society_type_pred', 'direccion':'state_pred'}, inplace = True)

In [37]:
df_preds.head()

,documento,company_name_csf,date_csf,notario_csf,society_type_csf,state_csf
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco


In [38]:
df_truth_compact.head()

,documento,company_name_pred,date_pred,notario_pred,society_type_pred,state_pred
0,10227,NaN,NaN,NaN,NaN,NaN
1,11821,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
2,12078,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
3,12192,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
4,12264,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"


In [39]:
df_merge = pd.merge(df_preds, df_truth_compact, on='documento', how='inner')
df_merge.head()

,documento,company_name_csf,date_csf,notario_csf,society_type_csf,state_csf,company_name_pred,date_pred,notario_pred,society_type_pred,state_pred
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco,exelencia ejecutiva premium,13/07/2011,rafael covarrubias flores,sa de cv,"guadalajara, jalisco"


### Comparo society_type
Uso Levenshtein distance. Cuenta la cantidad de ediciones que hacen falta para igualar los 2 strings. Si da 0 quiere decir que los strings son iguales. Cuanto más alto, más diferentes son.

In [40]:
def levenshtein_distance(a, b):
    """Return the Levenshtein edit distance between two strings *a* and *b*."""
    if a == b:
        return 0
    if pd.isnull(a) or pd.isnull(b):
        return np.nan
    if len(a) < len(b):
        a, b = b, a
    if not a:
        return len(b)
    previous_row = range(len(b) + 1)
    for i, column1 in enumerate(a):
        current_row = [i + 1]
        for j, column2 in enumerate(b):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (column1 != column2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1] 

Para que no se haga tan grande el df, armo uno nuevo con los matches

In [41]:
df_match = df_merge[['documento']].copy()
# df_match.head()

In [42]:
df_match['society_type_match'] = [levenshtein_distance(a, b) for a, b in zip(df_merge['society_type_csf'], df_merge['society_type_pred'])]
df_match['company_name_match'] = [levenshtein_distance(a, b) for a, b in zip(df_merge['company_name_csf'], df_merge['company_name_pred'])]
df_match['notario_match'] = [levenshtein_distance(a, b) for a, b in zip(df_merge['notario_csf'], df_merge['notario_pred'])]
df_match['state_match'] = [levenshtein_distance(a, b) for a, b in zip(df_merge['state_csf'], df_merge['state_pred'])]
df_match.head()

,documento,society_type_match,company_name_match,notario_match,state_match
0,11821,0,1,0.0,0.0
1,12078,0,0,0.0,7.0
2,12192,0,0,29.0,10.0
3,12264,0,0,NaN,18.0
4,12286,0,0,1.0,13.0


## Ahora falta estandarizar las fechas

## Usando la métrica de RapidFuzz
### (En lugar de Levenshtein)

In [43]:
from rapidfuzz import fuzz

In [44]:
df_match = df_merge[['documento']].copy()

In [55]:
N = df_match.shape[0]

for i in range(N):
    try:
        df_match.loc[i, 'society_type_match'] = fuzz.partial_ratio(df_merge.loc[i, 'society_type_csf'], df_merge.loc[i, 'society_type_pred'])
    except:
        df_match.loc[i, 'society_type_match'] = np.nan
        
    try:
        df_match.loc[i, 'company_name_match'] = fuzz.partial_ratio(df_merge.loc[i, 'company_name_csf'], df_merge.loc[i, 'company_name_pred'])
    except:
        df_match.loc[i, 'society_type_match'] = np.nan
        
    try:
        df_match.loc[i, 'notario_match'] = fuzz.partial_ratio(df_merge.loc[i, 'notario_csf'], df_merge.loc[i, 'notario_pred'])
    except:
        df_match.loc[i, 'society_type_match'] = np.nan
        
    try:
        df_match.loc[i, 'state_match'] = fuzz.partial_ratio(df_merge.loc[i, 'state_csf'], df_merge.loc[i, 'state_pred'])
    except:
        df_match.loc[i, 'society_type_match'] = np.nan
    
df_match.head()

,documento,society_type_match,company_name_match,notario_match,state_match
0,11821,100.0,90.909091,100.000000,100.000000
1,12078,100.0,100.000000,100.000000,100.000000
2,12192,100.0,100.000000,50.000000,100.000000
3,12264,NaN,100.000000,NaN,100.000000
4,12286,100.0,100.000000,95.833333,100.000000
5,12522,100.0,93.333333,28.571429,88.888889
6,12523,NaN,100.000000,NaN,100.000000
7,12543,NaN,100.000000,NaN,100.000000
8,12809,NaN,92.000000,NaN,85.714286
9,13174,100.0,100.000000,28.571429,100.000000


In [98]:
df_merge.head()

,documento,company_name_csf,date_csf,notario_csf,society_type_csf,state_csf,company_name_pred,date_pred,notario_pred,society_type_pred,state_pred
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco,exelencia ejecutiva premium,13/07/2011,rafael covarrubias flores,sa de cv,"guadalajara, jalisco"


## En qué páginas aparece cada campo

In [58]:
df_truth.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
1276,6631,10227,009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,6635,10227,019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,6908,11821,000,NaN,sa de cv,san luis potosi,21/07/2008,NaN,39413,NaN,mauricio mier padron,grupo cycme
95,6897,11821,011,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,6910,11821,015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_truth[df_truth["apoderado"].notnull()].head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
95,6897,11821,011,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,6930,12078,008,mauricio zúñiga de la cerda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,6943,12192,008,cesar higuera chavez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,6995,12264,011,lilia guadalupe gomez pezuela gamboa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,7003,12286,012,arturo uriel tavarez lópez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# list(df_truth.loc[df_truth["apoderado"].notnull(), 'pagina'])

In [96]:
# guardo las páginas de cada campo en un diccionario
paginas = {}

for column in df_truth.drop(['id', 'documento', 'pagina'], axis=1):
    paginas[column] = list(df_truth.loc[df_truth[column].notnull(), 'pagina'])
    
# paginas

In [95]:
# grafico las distribuciones
import plotly.express as px

for k in paginas.keys():
    try:
        fig = px.histogram(paginas[k], x=paginas[k], title=k, width=600, height=400)
        fig.show()
    except:
        continue

## Distribución de scores para cada campo

In [99]:
df_match.head()

,documento,society_type_match,company_name_match,notario_match,state_match
0,11821,100.0,90.909091,100.000000,100.0
1,12078,100.0,100.000000,100.000000,100.0
2,12192,100.0,100.000000,50.000000,100.0
3,12264,NaN,100.000000,NaN,100.0
4,12286,100.0,100.000000,95.833333,100.0


In [101]:
for column in df_match.drop(['documento'], axis=1):
    fig = px.histogram(df_match, x=column, title=column, width=600, height=400)
    fig.show()

## Top 5 matches por campo
### A los NaN los ignora. Queremos considerarlos como un mal match?
Si es así debería asignarles un 0.

In [109]:
df_match

,documento,society_type_match,company_name_match,notario_match,state_match
0,11821,100.0,90.909091,100.000000,100.000000
1,12078,100.0,100.000000,100.000000,100.000000
2,12192,100.0,100.000000,50.000000,100.000000
3,12264,NaN,100.000000,NaN,100.000000
4,12286,100.0,100.000000,95.833333,100.000000
5,12522,100.0,93.333333,28.571429,88.888889
6,12523,NaN,100.000000,NaN,100.000000
7,12543,NaN,100.000000,NaN,100.000000
8,12809,NaN,92.000000,NaN,85.714286
9,13174,100.0,100.000000,28.571429,100.000000


In [125]:
for column in df_match.drop(['documento'], axis=1):
#     print((df_match[['documento', column]].sort_values([column], ascending=[False])).iloc[:5])
#     print()
#     print(((df_match[['documento', column]].sort_values([column], ascending=[False])).iloc[:5])['documento'])
#     print()
    df_merge['documento']==((df_match[['documento', column]].sort_values([column], ascending=[False])).iloc[:5])['documento']

ValueError: Can only compare identically-labeled Series objects

In [123]:
df_merge.head()

,documento,company_name_csf,date_csf,notario_csf,society_type_csf,state_csf,company_name_pred,date_pred,notario_pred,society_type_pred,state_pred
0,11821,grupo cycmp,veintiumo de julio del dos mil ocho,mauricio mier padron,sa de cv,san luis potosi,grupo cycme,21/07/2008,mauricio mier padron,sa de cv,san luis potosi
1,12078,bitubi consulting,trece de febrero,jorge julian pereyra escudero,sa de cv,quintana roo,bitubi consulting,27/03/2018,jorge julian pereyra escudero,sa de cv,cancun quintana roo
2,12192,desarrollo de software monobits,22 de noviembre de 2012,yo,s de rl de cv,baja california,desarrollo de software monobits,21/01/2016,virgilio cesar valdez portales,s de rl de cv,"mexicali, baja california"
3,12264,charvel y pezuela group,dos de octubre,NaN,sa de cv,ciudad de mexico,charvel y pezuela group,02/10/2012,jorge rábago ordóñez,sa de cv,"ciudad de mexico, distrito federal"
4,12286,exelencia ejecutiva premium,NaN,rafael ovarrubias flores,sa de cv,jalisco,exelencia ejecutiva premium,13/07/2011,rafael covarrubias flores,sa de cv,"guadalajara, jalisco"
